## Loading the 20 newsgroups dataset

In [1]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Chargement de la liste des fichiers correspondant à ces catégories

In [3]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

________________________________________________________________________________
Cache loading failed
________________________________________________________________________________
No module named 'sklearn.utils._bunch'


#### Le dataset de retour est un scikit-learn bunch

### liste des noms de catégories demandés

In [4]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [5]:
len(twenty_train.data)

2257

In [6]:
len(twenty_train.filenames)

2257

In [7]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [8]:
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


In [9]:
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2], dtype=int64)

In [10]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


* Le prétraitement du texte, la segmentation et le filtrage des mots vides sont tous inclus dans CountVectorizer, qui crée un dictionnaire d'entités et transforme les documents en vecteurs d'entités
* CountVectorizer prend en charge le nombre de N-grammes de mots ou de caractères consécutifs. Une fois ajusté, le vectoriseur a construit un dictionnaire d'indices de caractéristiques
* N-grammes : le nombre de mot consideré comme un token (par ex : 2-grammes contient deux mots qui représente un token)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

* La valeur d'indice d'un mot dans le vocabulaire est liée à sa fréquence dans l'ensemble du corpus d'apprentissage.

In [12]:
count_vect.vocabulary_.get(u'algorithm')

4690

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

### Normalization : TfidfTransformer

* L'objectif d'utiliser tf-idf au lieu des fréquences brutes d'occurrence d'un jeton dans un document donné est de réduire l'impact des jetons qui se produisent très fréquemment dans un corpus donné et qui sont donc empiriquement moins informatifs que les caractéristiques qui se produisent dans un petite fraction du corpus de formation.

* Pour éviter les écarts potentiels, il suffit de diviser le nombre d'occurrences de chaque mot dans un document par le nombre total de mots dans le document : ces nouvelles fonctionnalités sont appelées **tf Term Frequencies**.

* La réduction d'échelle est appelée **tf–idf** pour « **Term Frequency times Inverse Document Frequency** ».

* **tf** et **tf–idf** peuvent être calculés comme suit en utilisant **TfidfTransformer**. 

* Le corpus est l'ensemble des documents 

In [14]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

In [15]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

### Inférence

In [16]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


### Création d'un Pipeline

* C'est un estimateur qui va contenir l'ensemble de nos traitements précedents : pretraitement, normalisation, et le model

In [17]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])

In [18]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [19]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.8348868175765646

### Création d'un nouveau Pipeline

In [ ]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                           max_iter=5, tol=None)),
 ])

text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

: 

* Affichage des valeurs des differentes metrics de classification

In [ ]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,
     target_names=twenty_test.target_names))

* Affichage de la matrice de confusion

In [ ]:
metrics.confusion_matrix(twenty_test.target, predicted)

### Utilisation de GridSearchCV

* La méthode GridSearchCV est utilisée pour obtenir les paramètres optimaux du model

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {
     'vect__ngram_range': [(1, 1), (1, 2)],
     'tfidf__use_idf': (True, False),
     'clf__alpha': (1e-2, 1e-3),
 }

In [ ]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

In [ ]:
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

* Inference

In [21]:
twenty_train.target_names[gs_clf.predict(['God is love'])[0]]

NameError: name 'gs_clf' is not defined

* Affichage du meilleur score

In [22]:
gs_clf.best_score_

NameError: name 'gs_clf' is not defined

* Affichage des paramètres optimaux

In [ ]:
for param_name in sorted(parameters.keys()):
     print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))